In [34]:
import pandas as pd
# read data
df_proposals_votes = pd.read_csv("./raw_data/2024-07-01_optimism_proposals_votes.csv")
# unify proposal_id to hexdecimal
df_proposals_votes.proposal_id = df_proposals_votes.proposal_id.apply(lambda x: str(hex(int(x[18:]))) if 'Proposal|Proposal|' in x else x)
# fill na otherwise groupby will drop rows
df_proposals_votes = df_proposals_votes.fillna('N/A')

/var/folders/db/bjx_3t8n2d73cp93pby9k97r0000gn/T/ipykernel_67797/4031484080.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_proposals_votes = pd.read_csv("./raw_data/2024-07-01_optimism_proposals_votes.csv")


In [42]:
df_summary.loc[df_summary['vote_type']=='OFF_CHAIN']

,vote_type,proposal_id,title,voted_choice,voting_power
0,OFF_CHAIN,0x5a35edacc5f2412fbcceb778c671a0a94b76b700812e...,Proposal C: Governance Fund Phase 1 - GYSR,AGAINST,1.0
1,OFF_CHAIN,0xf2c2621dce969b0f91b5cdf8270dfbde55f04ba5a8bd...,Proposal C: Governance Phase 1 - dHedge,FOR,1.0
2,OFF_CHAIN,0x037e4148cf8feb706c023108e1c82f9d765a6c6217dc...,Proposal B: Governance Phase 1 - Kromatika,AGAINST,1.0
3,OFF_CHAIN,0xb321a0cb19147a75ccca41d482594d785673b767593a...,Proposal H: Governance Phase 1 - Overnight.fi,AGAINST,1.0
4,OFF_CHAIN,0x0f3592ac3e437870affacc5fc64ce3c266b5041e24a3...,Proposal D: Governance Fund Phase 1 - Mean Fin...,FOR,1.0
...,...,...,...,...,...
1750326,OFF_CHAIN,0xa9457a5344747a6c272de786419fc7a4e83fcf0f927e...,Title: Special Voting Cycle #9b: Grants Counci...,MULTIPLE,1.0
1750327,OFF_CHAIN,0xa9457a5344747a6c272de786419fc7a4e83fcf0f927e...,Title: Special Voting Cycle #9b: Grants Counci...,MULTIPLE,1.0
1750328,OFF_CHAIN,0xa9457a5344747a6c272de786419fc7a4e83fcf0f927e...,Title: Special Voting Cycle #9b: Grants Counci...,MULTIPLE,1.0
1750329,OFF_CHAIN,0xa9457a5344747a6c272de786419fc7a4e83fcf0f927e...,Title: Special Voting Cycle #9b: Grants Counci...,MULTIPLE,1.0


In [44]:

df_summary = df_proposals_votes[['vote_type','proposal_id','title','voted_choice','voting_power']].copy()

df_summary.loc[df_summary['vote_type']=='OFF_CHAIN','voting_power'] = 1

df_summary.vote_type = df_summary.vote_type.map({'OFF_CHAIN':'citizen_house','ON_CHAIN':'token_house'})

df_summary = df_summary.groupby(['vote_type','proposal_id','title','voted_choice'],as_index=False).sum()

df_summary = df_summary.sort_values(by=['proposal_id', 'voting_power'],ascending=[True,False])

df_summary = df_summary.reset_index(drop=True)


In [45]:
len(df_summary.proposal_id.unique())

168

In [47]:
df_summary.to_csv('./voting_results_summary.csv')

# Token House Votes

In [2]:
# df_Top1000Delegates = pd.read_csv("./Top1000Delegates.csv")
# df_vote_cast = pd.read_csv("./vote_cast.csv")
# df_proposals = pd.read_csv("./proposals.csv")

In [153]:
df_onchain = df_proposals_votes.query('vote_type == "ON_CHAIN"').reset_index(drop=True)

# Step 1: Sort the DataFrame by 'proposal_id' and 'voting_power' in descending order
df_sorted = df_onchain.sort_values(by=['proposal_id', 'voting_power'], ascending=[True, False])

# Step 2: Group by 'proposal_id' and select the top 200 for each group
df_top_200_onchain = df_sorted.groupby('proposal_id').head(200).reset_index(drop=True)


In [154]:
df_top_200_onchain.to_csv("./token_house_proposals_votes.csv")

# Citizen House Votes

In [50]:
df_offchain = df_proposals_votes.query('vote_type == "OFF_CHAIN"').reset_index(drop=True)

# Step 1: Sort the DataFrame by 'proposal_id' and 'voting_power' in descending order
df_sorted = df_offchain.sort_values(by=['proposal_id', 'voting_power'], ascending=[True, False])

# Step 2: Group by 'proposal_id' and select the top 200 for each group
df_top_200_offchain = df_sorted.groupby('proposal_id').head(200).reset_index(drop=True)


In [51]:
df_top_200_offchain.to_csv('./citizen_house_proposals_votes.csv')

In [12]:
import requests
import pandas as pd

# df_citizen_snapshot_proposal = pd.read_json('citizen_snapshot_proposals.json')

url = "https://hub.snapshot.org/graphql?operationName=Proposals&query=query%20Proposals%20%7B%0A%20%20proposals%20(%0A%20%20%20%20first%3A%20100%2C%0A%20%20%20%20skip%3A%200%2C%0A%20%20%20%20where%3A%20%7B%0A%20%20%20%20%20%20space_in%3A%20%5B%22citizenshouse.eth%22%5D%2C%0A%20%20%20%20%20%20state%3A%20%22closed%22%0A%20%20%20%20%7D%2C%0A%20%20%20%20orderBy%3A%20%22created%22%2C%0A%20%20%20%20orderDirection%3A%20desc%0A%20%20)%20%7B%0A%20%20%20%20id%0A%20%20%20%20title%0A%20%20%20%20body%0A%20%20%20%20choices%0A%20%20%20%20start%0A%20%20%20%20end%0A%20%20%20%20snapshot%0A%20%20%20%20state%0A%20%20%20%20author%0A%20%20%20%20space%20%7B%0A%20%20%20%20%20%20id%0A%20%20%20%20%20%20name%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D"
response = requests.get(url)
df_citizen_snapshot_proposal=pd.DataFrame(response.json()['data']['proposals'])
df_citizen_snapshot_proposal=df_citizen_snapshot_proposal.rename(columns={'id':'proposal_id'})


In [15]:
df_citizen_snapshot_proposal_votes = pd.DataFrame()

for id in df_citizen_snapshot_proposal.proposal_id.to_list():
    url = f"https://hub.snapshot.org/graphql?operationName=Votes&query=query%20Votes%20%7B%0A%20%20votes%20(%0A%20%20%20%20first%3A%201000%0A%20%20%20%20skip%3A%200%0A%20%20%20%20where%3A%20%7B%0A%20%20%20%20%20%20proposal%3A%20%22{id}%22%0A%20%20%20%20%7D%0A%20%20%20%20orderBy%3A%20%22created%22%2C%0A%20%20%20%20orderDirection%3A%20desc%0A%20%20)%20%7B%0A%20%20%20%20id%0A%20%20%20%20voter%0A%20%20%20%20created%0A%20%20%20%20proposal%20%7B%0A%20%20%20%20%20%20id%0A%20%20%20%20%7D%0A%20%20%20%20choice%0A%20%20%20%20space%20%7B%0A%20%20%20%20%20%20id%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A"
    response = requests.get(url)
    temp = response.json()['data']['votes']
    for t in range(len(temp)):
        temp[t]['proposal_id'] = temp[t]['proposal']['id']
        temp[t]['space_id'] = temp[t]['space']['id']
    temp_df = pd.DataFrame(temp)
    df_citizen_snapshot_proposal_votes = pd.concat([df_citizen_snapshot_proposal_votes,temp_df])

df_citizen_snapshot_proposal_votes = pd.merge(df_citizen_snapshot_proposal_votes,df_citizen_snapshot_proposal,how="left",on='proposal_id')
df_citizen_snapshot_proposal_votes['proposal_id'] = df_citizen_snapshot_proposal_votes['id']


In [19]:
df_citizen_snapshot_proposal_votes

,id,voter,created,proposal,choice,space_x,proposal_id,space_id,title,body,choices,start,end,snapshot,state,author,space_y
0,0xe8db6c802becf16342b94d50b8ab5e7d552a67f1fed8...,0x4Fd6B3204A6Ab48978018b25821682F090840333,1719416248,{'id': '0xff9fcfd224f44cdcb6e8118cc5909e137604...,1,{'id': 'citizenshouse.eth'},0xe8db6c802becf16342b94d50b8ab5e7d552a67f1fed8...,citizenshouse.eth,Ratification of Round 4 Profit Definition,### Impact = profit\n\nOptimism’s Retroactive ...,"[Yes, ratify, No, don't consider profit at all]",1718906400,1719424800,20120885,closed,0x4ce779965d55f14BaF650f86758420BAE27AA3f1,"{'id': 'citizenshouse.eth', 'name': 'Citizens'..."
1,0x3803e3f6274d539741bd9fc5779ceebe9f8b3d3bde07...,0x1d3bf13f8f7A83390d03Db5E23A950778e1d1309,1719415021,{'id': '0xff9fcfd224f44cdcb6e8118cc5909e137604...,2,{'id': 'citizenshouse.eth'},0x3803e3f6274d539741bd9fc5779ceebe9f8b3d3bde07...,citizenshouse.eth,Ratification of Round 4 Profit Definition,### Impact = profit\n\nOptimism’s Retroactive ...,"[Yes, ratify, No, don't consider profit at all]",1718906400,1719424800,20120885,closed,0x4ce779965d55f14BaF650f86758420BAE27AA3f1,"{'id': 'citizenshouse.eth', 'name': 'Citizens'..."
2,0x9022e35782da34637c5fe29c142006b37fd1defc2c64...,0xDc0A92c350A52b6583e235a57901B8731aF8B249,1719414361,{'id': '0xff9fcfd224f44cdcb6e8118cc5909e137604...,1,{'id': 'citizenshouse.eth'},0x9022e35782da34637c5fe29c142006b37fd1defc2c64...,citizenshouse.eth,Ratification of Round 4 Profit Definition,### Impact = profit\n\nOptimism’s Retroactive ...,"[Yes, ratify, No, don't consider profit at all]",1718906400,1719424800,20120885,closed,0x4ce779965d55f14BaF650f86758420BAE27AA3f1,"{'id': 'citizenshouse.eth', 'name': 'Citizens'..."
3,0x2b201781f108bfb6726117b2454ab4e01a864888d55e...,0x55aEd0ce035883626e536254dda2F23a5b5D977f,1719407072,{'id': '0xff9fcfd224f44cdcb6e8118cc5909e137604...,1,{'id': 'citizenshouse.eth'},0x2b201781f108bfb6726117b2454ab4e01a864888d55e...,citizenshouse.eth,Ratification of Round 4 Profit Definition,### Impact = profit\n\nOptimism’s Retroactive ...,"[Yes, ratify, No, don't consider profit at all]",1718906400,1719424800,20120885,closed,0x4ce779965d55f14BaF650f86758420BAE27AA3f1,"{'id': 'citizenshouse.eth', 'name': 'Citizens'..."
4,0xe7f43dfa0d315b4f2f947bdb8675f0b9aaf56364b5e5...,0x5d36a202687fD6Bd0f670545334bF0B4827Cc1E2,1719397837,{'id': '0xff9fcfd224f44cdcb6e8118cc5909e137604...,2,{'id': 'citizenshouse.eth'},0xe7f43dfa0d315b4f2f947bdb8675f0b9aaf56364b5e5...,citizenshouse.eth,Ratification of Round 4 Profit Definition,### Impact = profit\n\nOptimism’s Retroactive ...,"[Yes, ratify, No, don't consider profit at all]",1718906400,1719424800,20120885,closed,0x4ce779965d55f14BaF650f86758420BAE27AA3f1,"{'id': 'citizenshouse.eth', 'name': 'Citizens'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,0xecd897f63118cbb4551e3e01265e82fdf93d6c7c8267...,0x66946def4ba6153C500D743b7A5FebfC1654d6bD,1706011546,{'id': '0x1552e2685f2f4560d9af1f69cba056d5a3e5...,1,{'id': 'citizenshouse.eth'},0xecd897f63118cbb4551e3e01265e82fdf93d6c7c8267...,citizenshouse.eth,Test Veto Vote,This vote is for Badgeholders to ensure they a...,[Veto],1705935623,1706540423,19063013,closed,0x47C88bb92B409fF25F6587EA611fac4e55f76007,"{'id': 'citizenshouse.eth', 'name': 'Citizens'..."
387,0xe1ff9d73d3b1f170851ee0c17658d188687c41c9c8fc...,0x0331969e189D63fBc31D771Bb04Ab44227D748D8,1706004292,{'id': '0x1552e2685f2f4560d9af1f69cba056d5a3e5...,1,{'id': 'citizenshouse.eth'},0xe1ff9d73d3b1f170851ee0c17658d188687c41c9c8fc...,citizenshouse.eth,Test Veto Vote,This vote is for Badgeholders to ensure they a...,[Veto],1705935623,1706540423,19063013,closed,0x47C88bb92B409fF25F6587EA611fac4e55f76007,"{'id': 'citizenshouse.eth', 'name': 'Citizens'..."
388,0x668ef61d845ffce4efd528a0cc8d7c923a922d02fdf3...,0x429F9aDA43e9F345CbB85EC88681BB70Df808892,1705997032,{'id': '0x1552e2685f2f4560d9af1f69cba056d5a3e5...,1,{'id': 'citizenshouse.eth'},0x668ef61d845ffce4efd528a0cc8d7c923a922d02fdf3...,c

In [5]:
df_citizen_snapshot_proposal_votes.to_csv('./citizen_house_proposals_votes2.csv')

In [86]:
# df_citizen_snapshot_proposal_votes.to_csv('./citizen_house_proposals_votes.csv')

# Farcaster

In [94]:
addresses = df_top_200_onchain.address.to_list()

In [ ]:
import requests

# Replace with your actual Farcaster API base URL
BASE_URL = "https://hub.pinata.cloud/v1/"

# List of Ethereum addresses to check
# addresses = [
#     "0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045",
#     "0x5e349eca2dc61abcd9dd99ce94d04136151a09ee",
#     # Add more addresses as needed
# ]

# Function to get Farcaster FID from Ethereum address
def get_farcaster_fid(address):
    response = requests.get(f"{BASE_URL}/verificationsByAddress", params={"address": address})
    if response.status_code == 200:
        data = response.json()
        if data.get("verifications"):
            return data["verifications"][0]["fid"]
    return None

# Function to get followers of a Farcaster account
def get_followers(fid):
    response = requests.get(f"{BASE_URL}/linksByFid", params={"fid": fid, "link_type": "follow"})
    if response.status_code == 200:
        data = response.json()
        return [msg["data"]["linkBody"]["targetFid"] for msg in data.get("messages", [])]
    return []

# Function to get followings of a Farcaster account
def get_followings(fid):
    response = requests.get(f"{BASE_URL}/linksByTargetFid", params={"target_fid": fid, "link_type": "follow"})
    if response.status_code == 200:
        data = response.json()
        return [msg["data"]["fid"] for msg in data.get("messages", [])]
    return []

# Main script
farcaster_accounts = {}

for address in addresses:
    fid = get_farcaster_fid(address)
    if fid:
        farcaster_accounts[address] = {
            "fid": fid,
            "followers": get_followers(fid),
            "followings": get_followings(fid)
        }
    else:
        farcaster_accounts[address] = None

# Print results
for address, data in farcaster_accounts.items():
    if data:
        print(f"Address: {address} (FID: {data['fid']})")
        print(f"Followers FIDs: {data['followers']}")
        print(f"Followings FIDs: {data['followings']}")
    else:
        print(f"Address: {address} has no Farcaster account.")


In [98]:
for address, data in farcaster_accounts.items():
    if data:
        print(f"Address: {address} (FID: {data['fid']})")
        print(f"Followers FIDs: {data['followers']}")
        print(f"Followings FIDs: {data['followings']}")
    # else:
    #     print(f"Address: {address} has no Farcaster account.")

In [ ]:
# https://crimson-tiny-urial-232.mypinata.cloud
# curl --request GET \
#   --url 'https://hub.pinata.cloud/v1/castsByFid?fid=6023&pageSize=10&reverse=true'
